In [3]:
import sagemaker, boto3, json
from sagemaker.session import Session

sagemaker_session = Session()

aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sess = sagemaker.Session()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [3]:
aws_region

'us-east-1'

# Preprocess Data

First, uploaded images to S3 from local storage. Then, we used the Data Wrangler to create an Image Resize Job to resize all images to the appropriate size for ResNet 50, (3,224,224). Next, we will turn the dataset into a recordIO, so that we can use the image classification algorithm. This step relies on the `preprocess.py` file found in the scripts folder.

In [4]:
# s3://lets-id-fish/fish-images-resized/Image-Resize-2023-09-14T13-41-28/image_col/test/

In [5]:
# from sagemaker.mxnet import MXNetProcessor
# from sagemaker.processing import ProcessingInput, ProcessingOutput

# mxp = MXNetProcessor(
#     framework_version='1.8.0',
#     py_version='py37',
#     role=role, 
#     instance_count=1,
#     instance_type='ml.t3.medium',
#     base_job_name='frameworkprocessor-mxnet'
# )

# BUCKET = 'lets-id-fish'
# S3_INPUT_PATH = 'fish-images-resized/Image-Resize-2023-09-14T13-41-28/image_col'


# processing_inputs = [
#     ProcessingInput(
#         input_name="input_img_data_train", 
#         source=f's3://{BUCKET}/{S3_INPUT_PATH}/train/',
#         destination="/opt/ml/processing/input/data/"
#     ),
#     ProcessingInput(
#         input_name="input_img_data_val", 
#         source=f's3://{BUCKET}/{S3_INPUT_PATH}/val/',
#         destination="/opt/ml/processing/input/data/"
#     ),
#     ProcessingInput(
#         input_name="input_img_data_test", 
#         source=f's3://{BUCKET}/{S3_INPUT_PATH}/test/',
#         destination="/opt/ml/processing/input/data/"
#     )
# ]

# processing_outputs = [
#     ProcessingOutput(
#         output_name="train", 
#         source="/opt/ml/processing/train",
#         destination=f"s3://{BUCKET}/{S3_INPUT_PATH}/recordIO/train"
#     ),
#     ProcessingOutput(
#         output_name="validation", 
#         source="/opt/ml/processing/validation",
#         destination=f"s3://{BUCKET}/{S3_INPUT_PATH}/recordIO/validation"
#     ),
#     ProcessingOutput(
#         output_name="test", 
#         source="/opt/ml/processing/test",
#         destination=f"s3://{BUCKET}/{S3_INPUT_PATH}/recordIO/test"
#     )
# ]

In [6]:
# mxp.run(
#     code="./scripts/preprocess.py",
#     inputs=processing_inputs,
#     outputs=processing_outputs,
#     arguments=[
#         "--input-s3-bucket", 
#         input_data.default_value,
#     ]
# )

Actually, just going to use the image files for now, but saving this for later!

# Fine-tune a pre-trained model on a custom dataset
***
The model available for fine-tuning attaches a classification layer to the corresponding feature extractor model available on TensorFlow/PyTorch hub, and initializes the layer parameters to random values. The output dimension of the classification layer
is determined based on the number of classes in the input data. The fine-tuning step fine-tunes the model parameters. The objective is to minimize classification error on the input data. The model returned by fine-tuning can be further deployed for inference. Below are the instructions for how the training data should be formatted for input to the model. 

- **Input:** A directory with as many sub-directories as the number of classes. 
    - Each sub-directory should have images belonging to that class in .jpg format. 
- **Output:** A trained model that can be deployed for inference. 
    - A label mapping file is saved along with the trained model file on the s3 bucket.  
    
The input directory should look like below if 
the training data contains images from two classes: roses and dandelion. The s3 path should look like
`s3://bucket_name/input_directory/`. Note the trailing `/` is required. The names of the folders and 'roses', 'dandelion', and the .jpg filenames
can be anything. The label mapping file that is saved along with the trained model on the s3 bucket maps the 
folder names 'roses' and 'dandelion' to the indices in the list of class probabilities the model outputs.
The mapping follows alphabetical ordering of the folder names. In the example below, index 0 in the model output list
would correspond to 'dandelion' and index 1 would correspond to 'roses'.

    input_directory
        |--roses
            |--abc.jpg
            |--def.jpg
        |--dandelion
            |--ghi.jpg
            |--jkl.jpg

***

### Retrieve Training artifacts
***
Here, for the selected model, we retrieve the training docker container, the training algorithm source, the pre-trained base model, and a python dictionary of the training hyper-parameters that the algorithm accepts with their default values. Note that the model_version="*" fetches the lates model. Also, we do need to specify the training_instance_type to fetch train_image_uri.
***

In [7]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

model_id, model_version = "pytorch-ic-resnet50", "*"
training_instance_type = "ml.p3.2xlarge"

# Retrieve the docker image
train_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    model_id=model_id,
    model_version=model_version,
    image_scope="training",
    instance_type=training_instance_type,
)
# Retrieve the training script
train_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="training"
)
# Retrieve the pre-trained model tarball to further fine-tune
train_model_uri = model_uris.retrieve(
    model_id=model_id, model_version=model_version, model_scope="training"
)

### Set Training parameters
***
Now that we are done with all the setup that is needed, we are ready to fine-tune our Image Classification model. To begin, let us create a [``sageMaker.estimator.Estimator``](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html) object. This estimator will launch the training job. 

There are two kinds of parameters that need to be set for training. 

The first one are the parameters for the training job. These include: (i) Training data path. This is S3 folder in which the input data is stored, (ii) Output path: This the s3 folder in which the training output is stored. (iii) Training instance type: This indicates the type of machine on which to run the training. Typically, we use GPU instances for these training. We defined the training instance type above to fetch the correct train_image_uri. 

The second set of parameters are algorithm specific training hyper-parameters.
***

In [8]:
# s3://lets-id-fish/fish-images-resized/Image-Resize-2023-09-14T13-41-28/image_col/test/

In [9]:
# Sample training data is available in this bucket
data_bucket = "lets-id-fish"
training_data_prefix = "SpeciesID-Images"

training_dataset_s3_path = f"s3://{data_bucket}/{training_data_prefix}/"

output_prefix = "Models"

s3_output_location = f"s3://{data_bucket}/{output_prefix}"

***
For algorithm specific hyper-parameters, we start by fetching python dictionary of the training hyper-parameters that the algorithm accepts with their default values. This can then be overridden to custom values.
***

In [10]:
from sagemaker import hyperparameters

# Retrieve the default hyper-parameters for fine-tuning the model
hyperparam = hyperparameters.retrieve_default(model_id=model_id, model_version=model_version)

# [Optional] Override default hyperparameters with custom values
print(hyperparam)

{'train_only_top_layer': 'True', 'epochs': '5', 'learning_rate': '0.001', 'batch_size': '4', 'reinitialize_top_layer': 'Auto'}


In [11]:
hyperparam['epochs'] = '5'
print(hyperparam)

{'train_only_top_layer': 'True', 'epochs': '5', 'learning_rate': '0.001', 'batch_size': '4', 'reinitialize_top_layer': 'Auto'}


In [12]:
hyperparam.update({'batch_size': '8'})

In [13]:
# hyperparameters.validate_hyperparameters(hyperparameters=hyperparam,model_id=model_id, 
#                                          model_version=model_version, sagemaker_session=sess)

### Train with Automatic Model Tuning ([HPO](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning.html)) <a id='AMT'></a>
***
Amazon SageMaker automatic model tuning, also known as hyperparameter tuning, finds the best version of a model by running many training jobs on your dataset using the algorithm and ranges of hyperparameters that you specify. It then chooses the hyperparameter values that result in a model that performs the best, as measured by a metric that you choose. We will use a [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html) object to interact with Amazon SageMaker hyperparameter tuning APIs.
***

In [14]:
from sagemaker.tuner import ContinuousParameter

# Use AMT for tuning and selecting the best model
use_amt = True

# Define objective metric per framework, based on which the best model will be selected.
metric_definitions_per_model = {
    "tensorflow": {
        "metrics": [{"Name": "val_accuracy", "Regex": "val_accuracy: ([0-9\\.]+)"}],
        "type": "Maximize",
    },
    "pytorch": {
        "metrics": [{"Name": "val_accuracy", "Regex": "val Acc: ([0-9\\.]+)"},
                   {"Name": "val_loss", "Regex": "val Loss: ([0-9\\.]+)"},
                   {"Name": "train_accuracy", "Regex": "train Acc: ([0-9\\.]+)"},
                   {"Name": "train_loss", "Regex": "train Loss: ([0-9\\.]+)"}],
        "type": "Maximize",
    },
}

# You can select from the hyperparameters supported by the model, and configure ranges of values to be searched for training the optimal model.(https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-define-ranges.html)
hyperparameter_ranges = {
    "adam-learning-rate": ContinuousParameter(0.0001, 0.1, scaling_type="Logarithmic")
}

# Increase the total number of training jobs run by AMT, for increased accuracy (and training time).
max_jobs = 6
# Change parallel training jobs run by AMT to reduce total training time, constrained by your account limits.
# if max_jobs=max_parallel_jobs then Bayesian search turns to Random.
max_parallel_jobs = 1

### Start Training
***
We start by creating the estimator object with all the required assets and then launch the training job.
***

In [15]:
from sagemaker.estimator import Estimator
from sagemaker.utils import name_from_base
from sagemaker.tuner import HyperparameterTuner

training_job_name = name_from_base(f"fish-id-{model_id}-transfer-learning")

# Create SageMaker Estimator instance
ic_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=train_model_uri,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparam,
    output_path=s3_output_location,
    base_job_name=training_job_name,
    enable_sagemaker_metrics=True
)

if use_amt:
    metric_definitions = next(
        value for key, value in metric_definitions_per_model.items() if model_id.startswith(key)
    )

    hp_tuner = HyperparameterTuner(
        ic_estimator,
        metric_definitions["metrics"][0]["Name"],
        hyperparameter_ranges,
        metric_definitions["metrics"],
        max_jobs=max_jobs,
        max_parallel_jobs=max_parallel_jobs,
        objective_type=metric_definitions["type"],
        base_tuning_job_name=training_job_name,
    )

    # Launch a SageMaker Tuning job to search for the best hyperparameters
    hp_tuner.fit({"training": training_dataset_s3_path})
else:
    # Launch a SageMaker Training job by passing s3 path of the training data
    ic_estimator.fit({"training": training_dataset_s3_path}, logs=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................!


In [16]:
from sagemaker import TrainingJobAnalytics

if use_amt:  # If using amt, select the model for the best training job.
    sage_client = boto3.Session().client("sagemaker")
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=hp_tuner._current_job_name
    )
    last_training_job_name = tuning_job_result["BestTrainingJob"]["TrainingJobName"]
else:
    last_training_job_name = ic_estimator._current_job_name
    
# Captured metrics can be accessed as a Pandas dataframe
df = TrainingJobAnalytics(training_job_name=last_training_job_name).dataframe()
df.head(10)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


,timestamp,metric_name,value
0,0.0,val_accuracy,0.24660
1,120.0,val_accuracy,0.42810
2,180.0,val_accuracy,0.46660
3,300.0,val_accuracy,0.45550
4,360.0,val_accuracy,0.46661
5,0.0,val_loss,3.70890
6,120.0,val_loss,2.67290
7,180.0,val_loss,2.64330
8,300.0,val_loss,2.60580
9,360.0,val_loss,2.64720


## Incrementally train the fine-tuned model

***
Incremental training allows you to train a new model using an expanded dataset that contains an underlying pattern that was not accounted for in the previous training and which resulted in poor model performance. You can use the artifacts from an existing model and use an expanded dataset to train a new model. Incremental training saves both time and resources as you don’t need to retrain a model from scratch.

One may use any dataset (old or new) as long as the dataset format remain the same (set of classes). Incremental training step is similar to the finetuning step discussed above with the following difference: In fine-tuning above, we start with a pre-trained model whereas in incremental training, we start with an existing fine-tuned model.
***

In [17]:
# Identify the previously trained model path based on the output location where artifacts are stored previously and the training job name.

if use_amt:  # If using amt, select the model for the best training job.
    sage_client = boto3.Session().client("sagemaker")
    tuning_job_result = sage_client.describe_hyper_parameter_tuning_job(
        HyperParameterTuningJobName=hp_tuner._current_job_name
    )
    last_training_job_name = tuning_job_result["BestTrainingJob"]["TrainingJobName"]
else:
    last_training_job_name = ic_estimator._current_job_name


last_trained_model_path = f"{s3_output_location}/{last_training_job_name}/output/model.tar.gz"

# last_trained_model_path = 's3://lets-id-fish/Models/fish-id-pytorch-ic-r-230911-2229-001-d615bcdd/output/model.tar.gz'

In [18]:
hyperparam.update({'epochs': '20'})

In [19]:
incremental_training_job_name = name_from_base(f"fish-id-{model_id}-incremental-training")
augmented_dataset_prefix = "species-id-images-augmented"
augmented_data_path = f"s3://{data_bucket}/{augmented_dataset_prefix}/"

incremental_train_estimator = Estimator(
    role=aws_role,
    image_uri=train_image_uri,
    source_dir=train_source_uri,
    model_uri=last_trained_model_path,
    entry_point="transfer_learning.py",
    instance_count=1,
    instance_type=training_instance_type,
    max_run=360000,
    hyperparameters=hyperparam,
    output_path=s3_output_location,
    base_job_name=incremental_training_job_name,
    enable_sagemaker_metrics=True
)

incremental_train_estimator.fit({"training": augmented_data_path}, logs=True)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


INFO:sagemaker:Creating training-job with name: fish-id-pytorch-ic-resnet50-incremental-2023-10-03-16-42-59-835


2023-10-03 16:42:59 Starting - Starting the training job......
2023-10-03 16:43:34 Starting - Preparing the instances for training......
2023-10-03 16:44:47 Downloading - Downloading input data......
2023-10-03 16:45:52 Training - Downloading the training image.....................
2023-10-03 16:49:23 Training - Training image download completed. Training in progress....bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2023-10-03 16:49:47,477 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2023-10-03 16:49:47,506 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2023-10-03 16:49:47,510 sagemaker_pytorch_container.training INFO     Invoking user training script.
2023-10-03 16:49:47,804 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "model": "

Once trained, we can use the same steps as in [Deploy & run Inference on the fine-tuned model](#4.5.-Deploy-&-run-Inference-on-the-fine-tuned-model) to deploy the model.

## Deploy & run Inference on the fine-tuned model
***
A trained model does nothing on its own. We now want to use the model to perform inference. For this example, that means predicting the class label of an image. We follow the same steps as in [3. Run inference on the pre-trained model](#3.-Run-inference-on-the-pre-trained-model). We start by retrieving the artifacts for deploying an endpoint. However, instead of base_predictor, we  deploy the `ic_estimator` that we fine-tuned.
***

In [5]:
aws_role

'arn:aws:iam::928680607065:role/SageMakerExecution'

In [6]:
from sagemaker import image_uris, model_uris, script_uris, hyperparameters

model_id, model_version = "pytorch-ic-resnet50", "*"

In [10]:
from sagemaker.model import Model
from sagemaker.utils import name_from_base

model_url = "s3://lets-id-fish/Models/fish-id-pytorch-ic-resnet50-incremental-2023-10-03-16-42-59-835/output/model.tar.gz"

In [12]:
inference_instance_type = "ml.p2.xlarge"

# Retrieve the inference docker container uri
deploy_image_uri = image_uris.retrieve(
    region=None,
    framework=None,
    image_scope="inference",
    model_id=model_id,
    model_version=model_version,
    instance_type=inference_instance_type,
)
# Retrieve the inference script uri
deploy_source_uri = script_uris.retrieve(
    model_id=model_id, model_version=model_version, script_scope="inference"
)

endpoint_name = name_from_base(f"fishid-endpoint-{model_id}-")

model = Model(image_uri=deploy_image_uri, 
              model_data=model_url, 
              role=aws_role)

# Use the estimator from the previous step to deploy to a SageMaker endpoint
finetuned_predictor = model.deploy(
    
    initial_instance_count=1,
    instance_type=inference_instance_type,
    entry_point="inference.py",
    image_uri=deploy_image_uri,
    source_dir=deploy_source_uri,
    endpoint_name=endpoint_name,
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------!

In [18]:
print(model)

---
Next, we download example images of a rose and a sunflower from the S3 bucket for inference.

---

In [ ]:
# s3://lets-id-fish/SpeciesID-Images-ForAWS-ByCommonFamily/test/Angelfishes/3852_287.jpg
# s3://lets-id-fish/SpeciesID-Images-ForAWS-ByCommonFamily/test/Wrasses/3889_8215.jpg

In [14]:
s3_bucket = "lets-id-fish"
key_prefix = "SpeciesID-Images"


def download_from_s3(images):
    for filename, image_key in images.items():
        boto3.client("s3").download_file(s3_bucket, f"{key_prefix}/{image_key}", filename)


fish_images = {
    "4242_626.jpg": "Acanthurus_chirurgus_/4242_626.jpg",
    "harlequinbass13.jpg": "Serranus_tigrinus_/harlequinbass13.jpg",
}
download_from_s3(fish_images)

---
Next, we query the fine-tuned model, parse the response and display the predictions.

---

In [21]:
from IPython.core.display import HTML
from sagemaker.serializers import IdentitySerializer

predictor = sagemaker.predictor.Predictor(endpoint_name=endpoint_name,sagemaker_session=sess)
predictor.serializer = IdentitySerializer("image/png")

for image_filename in fish_images.keys():
    with open(image_filename, "rb") as file:
        img = file.read()
    query_response = predictor.predict(
        img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    )
    # query_response = finetuned_predictor.predict(
    #     img, {"ContentType": "application/x-image", "Accept": "application/json;verbose"}
    # )
    model_predictions = json.loads(query_response)
    predicted_label = model_predictions["predicted_label"]
    display(
        HTML(
            f'<img src={image_filename} alt={image_filename} align="left" style="width: 250px;"/>'
            f"<figcaption>Predicted Label: {predicted_label}</figcaption>"
        )
    )

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received server error (0) from model with message "Your invocation timed out while waiting for a response from container model. Review the latency metrics for each container in Amazon CloudWatch, resolve the issue, and try again.". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/fishid-endpoint-pytorch-ic-resnet50--2023-10-04-17-45-37-804 in account 928680607065 for more information.

Sick! Inference is working!

---
Next, we clean up the deployed endpoint.

---

In [22]:
# Delete the SageMaker endpoint and the attached resources
finetuned_predictor.delete_model()
finetuned_predictor.delete_endpoint()

AttributeError: 'NoneType' object has no attribute 'delete_model'